# AutoClass

## AutoTokenizer

AutoTokenizer(分词器)负责将文本预处理为数字数组，作为模型的输入。有多个规则管理标记化过程，包括如何拆分单词以及应在什么级别拆分单词（在[标记器摘要](https://huggingface.co/docs/transformers/tokenizer_summary)中了解有关标记化的更多信息）。最重要的是要记住，您需要使用**相同的模型名称**实例化分词器，以确保您使用的是与预训练模型相同的分词规则。

In [1]:
from transformers import AutoTokenizer
model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [2]:
encode = tokenizer("We are very happy to show you the 🤗 Transformers library.")
print(encode)

{'input_ids': [101, 11312, 10320, 12495, 19308, 10114, 11391, 10855, 10103, 100, 58263, 13299, 119, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


- input_ids：你的token的数字表示。
- attention_mask：指示应注意哪些token。

AutoTokenizer(分词器)还可以接受输入列表，并填充和截断文本以返回具有统一长度的批次:

In [4]:
pt_batch = tokenizer(
    ["We are very happy to show you the 🤗 Transformers library.", "We hope you don't hate it."],
    padding=True,
    truncation=True,
    max_length=512,
    return_tensors="pt",
)

pt_batch

{'input_ids': tensor([[  101, 11312, 10320, 12495, 19308, 10114, 11391, 10855, 10103,   100,
         58263, 13299,   119,   102],
        [  101, 11312, 18763, 10855, 11530,   112,   162, 39487, 10197,   119,
           102,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0]])}

## AutoModel

Transformers 提供了一种简单而统一的方式来加载预训练实例。这意味着您可以像加载 AutoTokenizer 一样加载 AutoModel。唯一的区别是为任务选择正确的[AutoModel](https://huggingface.co/docs/transformers/v4.30.0/en/model_doc/auto#transformers.AutoModel)。对于文本（或序列）分类，您应该加载 AutoModelForSequenceClassification：

In [5]:
from transformers import AutoModelForSequenceClassification

model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
pt_model = AutoModelForSequenceClassification.from_pretrained(model_name)

现在将经过预处理的一批输入直接传递给模型。你只需要通过添加 ** 来解压字典：

In [7]:
pt_outputs = pt_model(**pt_batch)

该模型在logits属性中输出最终的激活值。对logits应用softmax函数来检索概率：

In [8]:
from torch import nn

pt_predictions = nn.functional.softmax(pt_outputs.logits, dim=-1)
print(pt_predictions)

tensor([[0.0021, 0.0018, 0.0115, 0.2121, 0.7725],
        [0.2084, 0.1826, 0.1969, 0.1755, 0.2365]], grad_fn=<SoftmaxBackward0>)


`nlptown/bert-base-multilingual-uncased-sentiment` 这是一个基于Bert-base的多语言无损检测模型，用于对六种语言的产品评论进行情感分析的微调： 英语、荷兰语、德语、法语、西班牙语和意大利语。它以星星的数量（1到5之间）预测评论的情绪。该模型旨在直接用作上述六种语言的产品评论的情感分析模型，或用于相关情感分析任务的进一步微调。